# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

In this lab you will: 

- Use ANOVA for testing multiple pairwise comparisons 
- Interpret results of an ANOVA and compare them to a t-test

## Load the data

Start by loading in the data stored in the file `'ToothGrowth.csv'`: 

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as scs
import itertools
from statsmodels.formula.api import ols

In [2]:
# Your code here
df = pd.read_csv('ToothGrowth.csv')
df.head()

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5


## Generate the ANOVA table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage:  

In [3]:
# Your code here
formula = 'len ~ C(supp) + dose'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

               sum_sq    df           F        PR(>F)
C(supp)    205.350000   1.0   11.446768  1.300662e-03
dose      2224.304298   1.0  123.988774  6.313519e-16
Residual  1022.555036  57.0         NaN           NaN


## Interpret the output

Make a brief comment regarding the statistics and the effect of supplement and dosage on tooth length: 

### Your comment here
H0: means are all equal vs HA: at least one mean is different
The p value of supp and dose are around 0 which is less than 0.05. Therefore, we reject the null hypothesis.
Therefore, there is at least one mean is different

## Compare to t-tests

Now that you've had a chance to generate an ANOVA table, its interesting to compare the results to those from the t-tests you were working with earlier. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterward, you'll conduct a t-test to compare the tooth length of these two different samples: 

In [4]:
# Your code here
df_OJ = df.loc[df['supp']=='OJ',['len']]
df_VC = df.loc[df['supp']=='VC',['len']]
OJ = np.array(df_OJ)
VC = np.array(df_VC)

Now run a t-test between these two groups and print the associated two-sided p-value: 

In [5]:
# Calculate the 2-sided p-value for a t-test comparing the two supplement groups
scs.ttest_ind(OJ,VC)
# The p value is 0.06 which is greater than 0.05. We fail to reject null hypothises
# Therefore, we can say they have the two group data are equal

Ttest_indResult(statistic=array([1.91526827]), pvalue=array([0.06039337]))

## A 2-Category ANOVA F-test is equivalent to a 2-tailed t-test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [6]:
# Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
formula = 'len ~ C(supp)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)
# Compare the p-value to that of the t-test above. 

# Now the p value from ANOVA F_test of the oj and vc supplement groups is 
# the same as the p value we got above from two sample t-test

# They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)

               sum_sq    df         F    PR(>F)
C(supp)    205.350000   1.0  3.668253  0.060393
Residual  3246.859333  58.0       NaN       NaN


## Run multiple t-tests

While the 2-category ANOVA test is identical to a 2-tailed t-test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [7]:
dict_group = {}
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = np.array(group[1])
    dict_group[group_name] = data
dict_group

{('OJ',
  0.5): array([15.2, 21.5, 17.6,  9.7, 14.5, 10. ,  8.2,  9.4, 16.5,  9.7]),
 ('OJ',
  1.0): array([19.7, 23.3, 23.6, 26.4, 20. , 25.2, 25.8, 21.2, 14.5, 27.3]),
 ('OJ',
  2.0): array([25.5, 26.4, 22.4, 24.5, 24.8, 30.9, 26.4, 27.3, 29.4, 23. ]),
 ('VC',
  0.5): array([ 4.2, 11.5,  7.3,  5.8,  6.4, 10. , 11.2, 11.2,  5.2,  7. ]),
 ('VC',
  1.0): array([16.5, 16.5, 15.2, 17.3, 22.5, 17.3, 13.6, 14.5, 18.8, 15.5]),
 ('VC',
  2.0): array([23.6, 18.5, 33.9, 25.5, 26.4, 32.5, 26.7, 21.5, 23.3, 29.5])}

While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2-sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [8]:
# Your code here; reuse your t-test code above to calculate the p-value for a 2-sided t-test
# for all combinations of the supplement-dose groups listed above. 
# (Since there isn't a control group, compare each group to every other group.)
dict_pvalue = {}
for i in itertools.combinations(dict_group.keys(),2):
    sample1 = dict_group[i[0]]
    #print(sample1)
    sample2 = dict_group[i[1]]
    #print(sample2)
    p = scs.ttest_ind(sample1,sample2)[1]
    dict_pvalue[i] = p
dict_pvalue

{(('OJ', 0.5), ('OJ', 1.0)): 8.357559281443774e-05,
 (('OJ', 0.5), ('OJ', 2.0)): 3.4018585295016214e-07,
 (('OJ', 0.5), ('VC', 0.5)): 0.005303661339923052,
 (('OJ', 0.5), ('VC', 1.0)): 0.04223992429368205,
 (('OJ', 0.5), ('VC', 2.0)): 7.025409196997986e-06,
 (('OJ', 1.0), ('OJ', 2.0)): 0.03736279585664383,
 (('OJ', 1.0), ('VC', 0.5)): 1.3372624230559434e-08,
 (('OJ', 1.0), ('VC', 1.0)): 0.0007807261651774468,
 (('OJ', 1.0), ('VC', 2.0)): 0.09583711277517494,
 (('OJ', 2.0), ('VC', 0.5)): 1.3381068810881244e-11,
 (('OJ', 2.0), ('VC', 1.0)): 2.3131084633597503e-07,
 (('OJ', 2.0), ('VC', 2.0)): 0.9637097790041267,
 (('VC', 0.5), ('VC', 1.0)): 6.492264598157612e-07,
 (('VC', 0.5), ('VC', 2.0)): 4.957285658438862e-09,
 (('VC', 1.0), ('VC', 2.0)): 3.397577925539582e-05}

## Summary

In this lesson, you implemented the ANOVA technique to generalize testing methods to multiple groups and factors.